In [48]:
from typing import Any, Type
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from datetime import datetime
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper

llm = ChatOpenAI(temperature=0.1)

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for"
    )

class SaveToFileToolArgsSchema(BaseModel):
    content: str = Field(
        description="The content to save to the file"
    )
    filename: str = Field(
        default="research_results",
        description="The filename where the content will be saved"
    )

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Try searching on Wikipedia,
    Extract content found from Wikipedia.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaAPIWrapper()
        return wikipedia.run(query)

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to search on DuckDuckGo.
    Try searching on DuckDuckGo,
    When DuckDuckGo finds a website, it goes into that website and extracts its content.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class SaveToFileTool(BaseTool):
    name = "SaveToFileTool"
    description = """
    Use this tool to save the text to a file.
    """

    args_schema: Type[
        SaveToFileToolArgsSchema
    ] = SaveToFileToolArgsSchema

    def _run(self, content: str, filename):
        filename_formatted = f"{filename}_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"
        file_path = f"./researchfiles/{filename_formatted}"
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
        return f"File saved {filename_formatted}"


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        SaveToFileTool(),
    ],
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Run the agent with this query: "{query}", the agent should try to search in Wikipedia or DuckDuckGo, if it finds a website in DuckDuckGo it should enter the website and extract it's content, then it should finish by saving the research to a .txt file.
            """,
        )
    ]
)

chain = {"query": RunnablePassthrough()} | prompt | agent

In [49]:
query = "Research about the XZ backdoor"
results = chain.invoke({"query":query})



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'Research about the XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.

Page: Supply chain attack
Summary: A supply chain attack is a cyber-attack that seeks to